In [6]:
import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging
import numpy as np
import blackboard
import ka
import ka_rp as karp
import ka_br as kabr
import database_generator as dg
import bb_basic
import time
import h5py
import train_surrogate_models as tm
import scipy.interpolate

In [16]:
ind, obj = dg.get_data(['height', 'smear', 'pu_content'], ['keff', 'void', 'doppler'])

In [30]:
sm = tm.Surrogate_Models()
model = 'ann'

sm.random = 0
sm.update_database(ind, obj)
sm.optimize_model(model)
print('R2 Score: {}'.format(sm.models[model]['score']))
print('MSE Score: {}'.format(sm.models[model]['mse_score']))
#print('CV Results: {}'.format(sm.models[model]['cv_results']))

R2 Score: 0.9106211494727335
MSE Score: 0.08937885052726642


In [31]:
print('Predicted: {}'.format(sm.predict(model, [[61.37,51.58,0.7340]])))
print('Predicted: {}'.format(sm.predict(model, [[59.72,50.01,0.8694]])))
print('Predicted: {}'.format(sm.predict(model, [[71.06,55.77,0.3536]])))

Predicted: [[   0.99941341 -113.84909307   -0.70747017]]
Predicted: [[   0.99826766 -110.86682936   -0.72453021]]
Predicted: [[   1.00595744 -117.80498917   -0.6638631 ]]


In [17]:
sm2 = {}
design_var, objective_func = np.asarray(ind), np.asarray(obj)
for num, objective in enumerate(['keff', 'void', 'doppler']):
    sm2[objective] = scipy.interpolate.LinearNDInterpolator(design_var, objective_func[:,num]) 

In [18]:
x_list = [[61.37,51.58,0.7340],[59.72,50.01,0.8694],[71.06,55.77,0.3536]]
for design in x_list:
    x = []
    for interpolator in sm2.values():
        x.append(float(interpolator(tuple(design))))
    print(x)

[0.9992587833657331, -116.3956786137983, -0.7162576521744397]
[0.9986864120000001, -111.95413008061794, -0.7529935737270184]
[1.0063239933333332, -119.45645329200596, -0.6373027564232986]


In [3]:
ns = run_nameserver()
bb = run_agent(name='bb', base=bb_basic.BbTraditional)

bb.connect_agent('rp', 'ka_rp')
bb.connect_agent('br', 'ka_br')

bb.add_abstract_lvl(1, {'valid': bool})
bb.add_abstract_lvl(2, {'reactor parameters': {'height': float, 'smear': float, 'pu_content': float, 'keff': float, 'void_coeff': float, 'doppler_coeff': float}})
#bb.load_h5()

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:16801 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:16801
INFO [2020-04-14 18:34:09.242815] (bb): Connected agent ka_rp of agent type rp
INFO [2020-04-14 18:34:09.281582] (bb): Connected agent ka_br of agent type br
NS shut down.


In [ ]:
while not bb.get_attr('_complete'):
    bb.publish_trigger()
    bb.controller()
    bb.send_executor()
    bb.wait_for_ka()
    print('Finished trigger event: {}'.format(bb.get_attr('_trigger_event')))

print(bb.get_attr('abstract_lvls')['level 1'])

In [ ]:
h5 = h5py.File('bb_archive.h5', 'r')
for k,v in h5['level 2']['core_[67.38, 57.01, 0.42]']['reactor parameters'].items():
    print(k,v[0])
h5.close()